# SBA 빅데이터 사이언스 실무연계 프로젝트 🏫

## Machine Learning sevice on DSVM ☁️ - 두 번째
---
### 02. Cifar10 Image Classification with Keras on Azure ML service
* [Azure Machine Learning이란?](https://docs.microsoft.com/ko-kr/azure/machine-learning/service/overview-what-is-azure-ml/?WT.mc_id=AI-MVP-5003262)  
* [Azure Machine Learning을 사용하여 모델 학습](https://docs.microsoft.com/ko-kr/azure/machine-learning/service/concept-train-machine-learning-model/?WT.mc_id=AI-MVP-5003262)

##### 전미정 | 2019.11.21. | ninevincentg@gmail.com
---

## Part1. 개발 환경 설정

1. Workspace 연결
2. Experiment 생성
3. 학습에 사용할 remote compute 생성
4. 데이터 준비

### 1-1. Workspace 연결

In [ ]:
import azureml
from azureml.core import Workspace

# Azure core SDK version 확인
print("Azure ML SDK Version: ", azureml.core.VERSION)

In [ ]:
# 다운 받은 config.json에서 복붙해주세요!

subscription_id = 'subscription_id'
resource_group  = 'resource_group'
workspace_name  = 'workspace_name'

try:
    ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
    ws.write_config()
    print('Library configuration succeeded')
except:
    print('Workspace not found')
    
# 계정 인증문구가 뜨면 하단에 영문과 숫자로 이루어진 코드 9자리를 복사한 후 링크(https://microsoft.com/devicelogin)을 클릭해 붙여넣어주세요.

### 1-2. Experiment 생성

In [ ]:
from azureml.core import Experiment

exp = Experiment(workspace=ws, name='keras_cifar10')

### 1-3. 학습에 사용할 remote compute 생성 🌟⭐️🌟

**참고자료**  
[Azure Machine Learning에서 계산 대상 이란?](https://docs.microsoft.com/ko-kr/azure/machine-learning/service/concept-compute-target/?WT.mc_id=AI-MVP-5003262)

In [ ]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

# cluster의 이름 설정
# compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "cpucluster")
# # cluster의 최소/최대 노드 수 설정
# compute_min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 0)
# compute_max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 4)

# # 기본 VM size: STANDARD_D2_V2
# vm_size = os.environ.get("AML_COMPUTE_CLUSTER_SKU", "STANDARD_D2_V2")

# # ----- GPU를 사용하려면 아래 4줄 코드를 주석 해제해주세요. -----#
compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "gpucluster")
compute_min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 0)
compute_max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 4)

vm_size = os.environ.get("AML_COMPUTE_CLUSTER_SKU", "STANDARD_NC12")


if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('found compute target. just use it. ' + compute_name)
else:
    print('creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                                min_nodes = compute_min_nodes, 
                                                                max_nodes = compute_max_nodes)

    # compute 생성
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)
    
    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    

In [ ]:
# Remote compute 목록 및 선택된 compute 확인 
compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, ct.type, ct.provisioning_state)
    
print("Selected computer: ", compute_target.get_status().serialize()['vmSize'])

### 1-4. 데이터 준비(Blob storage 연결)

In [ ]:
# 데이터 다운로드
import urllib

os.makedirs('./data/', exist_ok=True)

images_url = 'https://www.dropbox.com/s/ua5rtnb1k4mdzpz/dataset.npy?dl=1'
label_url = 'https://www.dropbox.com/s/9apie8xg9vqylws/label.npy?dl=1'

urllib.request.urlretrieve(images_url, filename='./data/images.npy')
urllib.request.urlretrieve(label_url, filename='./data/label.npy')

In [ ]:
# 데이터 확인
from sklearn.model_selection import train_test_split
import numpy as np

images = np.load('./data/images.npy')
label = np.load('./data/label.npy')

X_train, X_test, y_train, y_test = train_test_split(images, label, test_size = 0.2)

print("Total Images:", images.shape)
print("Total Label:", label.shape)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape, sep='\n')

In [ ]:
# 이미지, 레이블 확인

%matplotlib inline

import matplotlib.pyplot as plt

cifar10_label = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

count = 0
sample_size = 10
plt.figure(figsize = (16, 6))
for i in np.random.permutation(images.shape[0])[:sample_size]:
    count = count + 1
    plt.subplot(1, sample_size, count)
    plt.axhline('')
    plt.axvline('')
    plt.text(x=10, y=-10, s=cifar10_label[label[i]], fontsize=15)
    plt.imshow(images[i], cmap=plt.cm.Greys)

plt.show()

In [ ]:
# Workspace와 연결된 datastore 가져오기 🌟⭐️🌟

ds = ws.get_default_datastore()
print(ds)

In [ ]:
# datastore에 이미지 파일 업로드 🌟⭐️🌟

ds.upload(src_dir='./data', target_path='cifar10', overwrite=True, show_progress=True)

---
## Part2. 모델 학습 환경 설정 🌟⭐️🌟

1. 원격 리소스로 전달할 파일 디렉토리 생성
2. 학습 스크립트 설정
3. Estimator 생성 및 스크립트 파라미터 설정
4. Estimator 제출

**참고자료**  
[Azure Machine Learning에서 MNIST 데이터와 scikit-learn을 사용하여 이미지 분류 모델 학습](https://docs.microsoft.com/ko-kr/azure/machine-learning/service/tutorial-train-models-with-aml/?WT.mc_id=AI-MVP-5003262)

### 2-1. 디렉토리 생성

In [ ]:
import os
import shutil

# 스크립트를 저장할 디렉토리 생성
script_folder = './keras_cifar10'
os.makedirs(script_folder, exist_ok=True)

# 학습 스크립트 복사
shutil.copy('./cifar10_train.py', script_folder)

### 2-2. 학습 스크립트 복사

In [ ]:
# 학습 스크립트 복사 및 확인
with open(os.path.join(script_folder, './cifar10_train.py'), 'r') as f:
    print(f.read())

### 2-3. Estimator 생성 및 파라미터 설정

In [ ]:
from azureml.train.dnn import TensorFlow

script_params = {
    '--data-folder': ds.path('cifar10').as_mount(),
    '--batch-size': 32,
    '--epoch' : 50,
    '--first-dropout': 0.2,
    '--second-dropout': 0.5,
    '--learning-rate': 0.01
}

est = TensorFlow(source_directory=script_folder,
                 script_params=script_params,
                 compute_target=compute_target, 
                 pip_packages=['keras', 'matplotlib', 'sklearn'],
                 entry_script='cifar10_train.py', 
                 use_gpu=True)

### 2-4. Estimator 제출

In [ ]:
run = exp.submit(est)

---
## Part3. 원격 학습 모니터링
`run` 객체 결과 보기

### 3-1. Jupyter Widget으로 실행 상태 확인

In [ ]:
from azureml.widgets import RunDetails
RunDetails(run).show()

### 3-2. 완료시 로그 확인

In [ ]:
run.wait_for_completion(show_output=True)

In [ ]:
run.get_details()

In [ ]:
run.get_file_names()

---
## Part4. 모델 불러오기 및 테스트

### 4-1. 클러스터에 저장된 모델 내려받기

In [ ]:
# create a model folder in the current directory
os.makedirs('./model', exist_ok=True)

for f in run.get_file_names():
    if f.startswith('outputs/model'):
        output_file_path = os.path.join('./model', f.split('/')[-1])
        print('Downloading from {} to {} ...'.format(f, output_file_path))
        run.download_file(name=f, output_file_path=output_file_path)

### 4-2. 로컬 테스트

In [ ]:
import keras
import tensorflow as tf

print("Keras version:", keras.__version__)
print("Tensorflow version:", tf.__version__)

In [ ]:
# 저장된 모델 디스크에 불러오기
from keras.models import model_from_json

json_file = open('model/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

loaded_model.load_weights("model/model.h5")
print("Model loaded from disk.")

In [ ]:
# X_train/y_train 로컬 테스트
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])


y_test_ = np.eye(10)[y_test.reshape(-1)]
for index in np.random.choice(len(y_test_), 5, replace = False):
    predicted = loaded_model.predict(X_test[index:index + 1]/255.0)[0]
    label = y_test_[index]
    result_label = np.where(label == np.amax(label))
    result_predicted = np.where(predicted == np.amax(predicted))
    
    title = "Label value = %s  Predicted value = %s " % (cifar10_label[result_label[0][0]],  cifar10_label[result_predicted[0][0]])
    fig = plt.figure(1, figsize = (3,3))
    ax1 = fig.add_axes((0,0,.8,.8))
    ax1.set_title(title)
    images = X_test
    plt.imshow(images[index], cmap = plt.cm.gray_r, interpolation = 'nearest')
    plt.show()

## Part5. AML에 모델 등록하기

In [ ]:
# 작업 수행 ID 확인
print(run.id)

In [ ]:
# 실험에서 생성된 모델 파일 Azure Machine Learning Service에 등록
model = run.register_model(model_name = 'keras_cifar10_gpu', model_path = 'outputs/model')

---
🥳